Importing neccesary libraries


In [ ]:
from langchain_community.document_loaders.text import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from transformers import pipeline
from langchain_core.vectorstores import VectorStoreRetriever
from langchain.chains import RetrievalQA
from langchain_huggingface.llms import HuggingFacePipeline

Loading the data 

In [ ]:
loader = TextLoader("harry1.txt")
docs = loader.load_and_split(text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=15))

Getting the Hugging Face Embedding and creating the vector database

In [ ]:
embed_model = HuggingFaceEmbeddings(model_name="intfloat/e5-large-v2")

vector_store = Chroma.from_documents(docs, embedding=embed_model, persist_directory="./chroma_db4")

c:\Users\acer\Desktop\programming\projects\RAG\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


Checking how well does the similarity search work on our vector database

In [4]:
vector_store.similarity_search("What section of Hogwarts does the Sorting Hat ultimately place Harry Potter in?", k=3)[0].page_content

"Your top hats sleek and tall,\n\nFor I'm the Hogwarts Sorting Hat\n\nAnd I can cap them all.\n\nThere's nothing hidden in your head\n\nThe Sorting Hat can't see,\n\nSo try me on and I will tell you\n\nWhere you ought to be.\n\nYou might belong in Gryffindor,\n\nWhere dwell the brave at heart,\n\nTheir daring, nerve, and chivalry Set Gryffindors apart;\n\nYou might belong in Hufflepuff,\n\nWhere they are just and loyal,\n\nThose patient Hufflepuffis are true And unafraid of toil;\n\nOr yet in wise old Ravenclaw,"

Creating the retriever part of the RAG

In [ ]:
retriever = VectorStoreRetriever(vectorstore=vector_store,search_kwargs = {'k':1})
generator = pipeline(
    "text2text-generation",         
    model="google/flan-t5-base",
    max_new_tokens=512
)
llm = HuggingFacePipeline(pipeline=generator)

# Build the retrieval QA chain
qa_chain = RetrievalQA.from_llm(llm=llm, retriever=retriever)


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Device set to use cpu


Quiz time

In [ ]:
qa_chain.invoke("""13.Who was the headmaster of Hogwarts?""")

{'query': '13. Give general information: Who was the headmaster of Hogwarts?\n\n',
 'result': 'Albus Dumbledore.'}

In [14]:
import gradio as gr

def qa_interface(question):
    return qa_chain.invoke(question)['result']

gr.Interface(
    fn=qa_interface,
    inputs=gr.Textbox(label="Ask a question about Harry Potter"),
    outputs=gr.Textbox(label="Answer"),
    title="Harry Potter QA"
).launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
